In [1]:
!pip install transformers datasets torch


  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl.metadata (1.8 kB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinu

In [2]:
!pip install transformers
!pip install torch

from transformers import BartForConditionalGeneration, BartTokenizer
import torch


In [3]:
import json
import pandas as pd
from datasets import Dataset

# Load the JSON data
with open('intentc.json') as f:
    data = json.load(f)

# Convert to pandas DataFrame
df = pd.DataFrame(data)

# Create a Dataset object
dataset = Dataset.from_pandas(df)


In [4]:
from transformers import BartTokenizer

# Load the tokenizer
tokenizer = BartTokenizer.from_pretrained('facebook/bart-base')

# Tokenize the dataset
def preprocess_function(examples):
    inputs = examples["Question"]
    targets = examples["Query"]
    model_inputs = tokenizer(inputs, max_length=512, truncation=True)

    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=512, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_dataset = dataset.map(preprocess_function, batched=True)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.72k [00:00<?, ?B/s]

Map:   0%|          | 0/1200 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:4016: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


In [5]:
from transformers import BartForConditionalGeneration, Trainer, TrainingArguments, DataCollatorForSeq2Seq

# Load the model
model = BartForConditionalGeneration.from_pretrained('facebook/bart-base')

# Define training arguments
training_args = TrainingArguments(
    output_dir='./results',
    eval_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    num_train_epochs=3,
    weight_decay=0.01,
)

# Create a data collator
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    eval_dataset=tokenized_dataset,
    data_collator=data_collator,
)

# Train the model
trainer.train()


model.safetensors:   0%|          | 0.00/558M [00:00<?, ?B/s]

Epoch,Training Loss,Validation Loss
1,0.329200,0.005957
2,0.023300,0.000012
3,0.009600,0.000005


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'early_stopping': True, 'num_beams': 4, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0, 'forced_eos_token_id': 2}
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'early_stopping': True, 'num_beams': 4, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0, 'forced_eos_token_id': 2}
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file

TrainOutput(global_step=1800, training_loss=0.10126233849260542, metrics={'train_runtime': 247.5487, 'train_samples_per_second': 14.543, 'train_steps_per_second': 7.271, 'total_flos': 28423007907840.0, 'train_loss': 0.10126233849260542, 'epoch': 3.0})

In [6]:
# Save the model
model.save_pretrained("./fine-tuned-bart")
tokenizer.save_pretrained("./fine-tuned-bart")


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'early_stopping': True, 'num_beams': 4, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0, 'forced_eos_token_id': 2}


('./fine-tuned-bart/tokenizer_config.json',
 './fine-tuned-bart/special_tokens_map.json',
 './fine-tuned-bart/vocab.json',
 './fine-tuned-bart/merges.txt',
 './fine-tuned-bart/added_tokens.json')

In [24]:
from transformers import BartTokenizer, BartForConditionalGeneration

# Load the fine-tuned model and tokenizer
model = BartForConditionalGeneration.from_pretrained('./fine-tuned-bart')
tokenizer = BartTokenizer.from_pretrained('./fine-tuned-bart')

# Function to generate MongoDB query
def generate_query(question):
    inputs = tokenizer(question, return_tensors='pt', max_length=512, truncation=True, padding="max_length")
    outputs = model.generate(**inputs, max_length=512, num_beams=4, early_stopping=True)
    query = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return query

# Example usage
question = "What are all the details of John Kerr?"
query = generate_query(question)
print(query)


db.patients.find({"name.first": "John", "name.last": "Kerr"}, {"_id": 0})


In [16]:
!pip install nltk rouge-score


  Preparing metadata (setup.py) ... done
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=83094bab01a10cf353aad18facd173e65e16351883824d5d83da162b86788ff3
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge-score


In [17]:
import nltk
from nltk.translate.bleu_score import sentence_bleu
from rouge_score import rouge_scorer
from transformers import BartTokenizer, BartForConditionalGeneration

# Load the fine-tuned model and tokenizer
model = BartForConditionalGeneration.from_pretrained('./fine-tuned-bart')
tokenizer = BartTokenizer.from_pretrained('./fine-tuned-bart')

def generate_query_finetuned(question):
    inputs = tokenizer(question, return_tensors='pt', max_length=512, truncation=True, padding="max_length")
    outputs = model.generate(**inputs, max_length=512, num_beams=4, early_stopping=True)
    query = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return query


In [18]:
test_data = [

    {'input': 'Which patients are scheduled for surgery tomorrow?', 'expected_query': '{"surgery_date": "2024-07-29"}'},
    {'input': 'What appointments does Grace Smith have scheduled?', 'expected_query': '{"db.patients.find({ "name.first": "Grace", "name.last": "Smith" }, { "appointments": 1, "_id": 0"}'},
    {'input': 'What surgeries has Charlie Smith had in the past?', 'expected_query': '{"db.patients.find({ "name.first": "Charlie", "name.last": "Smith" }, { "medical_history.surgeries": 1, "_id": 0"}'},
    {'input': 'What is the insurance provider for David Smith?', 'expected_query': '{"db.patients.find({ "name.first": "David", "name.last": "Smith" }, { "insurance.provider": 1, "_id": 0"}'},
    {'input': 'What is the marital status of Charlie Smith?', 'expected_query': '{"db.patients.find({ "name.first": "Charlie", "name.last": "Smith" }, { "marital_status": 1, "_id": 0"}'},
    {'input': 'What is the contact phone number for Eve Smith?', 'expected_query': '{"db.patients.find({ "name.first": "Eve", "name.last": "Smith" }, { "contact.phone": 1, "_id": 0"}'},
    {'input': 'What surgeries has Olivia Smith had in the past?', 'expected_query': '{"db.patients.find({ "name.first": "Olivia", "name.last": "Smith" }, { "medical_history.surgeries": 1, "_id": 0"}'},
    {'input': 'How many patients have a blood group A+?','expected_query':'{"db.patients.find({ "blood_group": "A+" }, { "name": 1, "id": 1, "age": 1, "blood_group": 1, "_id": 0 }'},
    {'input': 'What are all the details of Sarah Kennedy?','expected_query':'{"db.patients.find({ "name.first": "Sarah", "name.last": "Kennedy" }, { "_id": 0 }'},
    {'input': 'How many patients have a blood group B-?','expected_query':'{"db.patients.find({ "blood_group": "B-" }, { "name": 1, "id": 1, "age": 1, "blood_group": 1, "_id": 0 }'}
]


In [19]:
# Initialize metrics
bleu_scores = []
rouge_scores = {'rouge1': [], 'rouge2': [], 'rougeL': []}
precisions = []

# Initialize the ROUGE scorer
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)


In [20]:
!pip install nltk rouge-score
import nltk
from nltk.translate.bleu_score import sentence_bleu
from rouge_score import rouge_scorer

# Download the punkt tokenizer models
nltk.download('punkt')



[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [21]:
# Evaluation
for data in test_data:
    generated_query = generate_query_finetuned(data['input'])

    # Calculate BLEU score
    reference = nltk.word_tokenize(data['expected_query'])
    hypothesis = nltk.word_tokenize(generated_query)
    bleu_score = sentence_bleu([reference], hypothesis)
    bleu_scores.append(bleu_score)

    # Calculate ROUGE scores
    rouge_score = scorer.score(data['expected_query'], generated_query)
    rouge_scores['rouge1'].append(rouge_score['rouge1'].fmeasure)
    rouge_scores['rouge2'].append(rouge_score['rouge2'].fmeasure)
    rouge_scores['rougeL'].append(rouge_score['rougeL'].fmeasure)

    # Calculate Precision
    reference_set = set(reference)
    hypothesis_set = set(hypothesis)
    true_positives = len(reference_set.intersection(hypothesis_set))
    precision = true_positives / len(hypothesis_set) if hypothesis_set else 0
    precisions.append(precision)


/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)


In [22]:
# Average metrics
avg_bleu = sum(bleu_scores) / len(bleu_scores)
avg_rouge1 = sum(rouge_scores['rouge1']) / len(rouge_scores['rouge1'])
avg_rouge2 = sum(rouge_scores['rouge2']) / len(rouge_scores['rouge2'])
avg_rougeL = sum(rouge_scores['rougeL']) / len(rouge_scores['rougeL'])
avg_precision = sum(precisions) / len(precisions)

print(f'Average BLEU Score: {avg_bleu}')
print(f'Average ROUGE-1 Score: {avg_rouge1}')
print(f'Average ROUGE-2 Score: {avg_rouge2}')
print(f'Average ROUGE-L Score: {avg_rougeL}')
print(f'Average Precision: {avg_precision}')


Average BLEU Score: 0.7316622924541205
Average ROUGE-1 Score: 0.8420012449424213
Average ROUGE-2 Score: 0.7809298642533936
Average ROUGE-L Score: 0.8364456893868658
Average Precision: 0.8063954002653693
